Task 1
This file contains the functions to assess the risk of investing a certain stock by comparing its amplitudes in the past year with the stop-loss we set. Besides, it also contains a function that selects all stocks whose amplitudes are less than the stop-loss.

In [6]:
#import necessary modules that we need 
from jqdatasdk import *
auth("18639879991","Mzy1208@3")
import pandas as pd
from datetime import datetime, timedelta

auth success 


In [3]:
#return the amplitude of a stock over some time
#@param: scu name of the stock
#@param: end_date end date of investment
#@param: period length of each period(num of days)
#@param: count number of periods we want to check(default to 1)
#@return: the amplitude of a stock over the most recent investment period
def get_amplitude(scu,end_date,period,count=1):
    fqy=period+"d"
    df1=get_price(security=scu,count=count,end_date=end_date,fields=['high','low'],frequency=fqy)
    df1['h-l']=df1['high']-df1['low']
    last_end_date=(datetime.strptime(end_date, '%Y-%m-%d')-timedelta(days=int(period))).strftime('%Y-%m-%d')
    df2=get_price(security=scu,count=count,end_date=last_end_date,fields=['close'],frequency=fqy)
    df2['close'].index=df1['h-l'].index
    amplitude=((df1['h-l']/df2['close']).tolist())[0]
    return amplitude

In [ ]:
#return a list storing amplitudes of a stock in the past year
#@param: scu name of the stock
#@param: end_state end date of the investment
#@param: period length if each period(num of days)
#@return: a list storing amplitudes of a stock in the past year
def return_a_list(scu,end_date,period):
    list1=[]
    list1.append(get_amplitude(scu,end_date,period,count=1))
    for i in range(1,365):
        end_date=(datetime.strptime(end_date, '%Y-%m-%d')-timedelta(days=1)).strftime('%Y-%m-%d')
        amplitude=get_amplitude(scu,end_date,period,count=1)
        list1.append(amplitude)
    sorted_list=sorted(list1)
    return sorted_list

In [ ]:
#compute the probability that a stock's amplitude in the next period is higher than stop loss based on its past performance
#@param: sorted_list a list storing the amplitudes of a stock over the past year
#@param stop_loss the stop loss set by the consumer
#@return the probability that a stock's amplitude in the next period is higher than stop loss
def compute_prob(sorted_list,stop_loss):  
    count=0
    for i in sorted_list:
        if i < stop_loss:
            count=count+1
        else:
            break
    prob=(len(sorted_list)-count)/len(sorted_list)
    return prob

In [ ]:
#compute the risk of investing a stock
#@param: scu name of stock
#@param: start_date start date of the investment
#@param: period length of a period(num of days)
#@param: stop_loss stop loss that the consumer sets
#@return risk fo investing a stock over one period
def assess_risk(scu,start_date,period,stop_loss):
    sorted_list=return_a_list(scu, start_date, period, stop_loss)
    risk=compute_prob(sorted_list,stop_loss)
    return risk

In [ ]:
#take 000010.XSHE as an example, starting to invest on 2021-09-30 for 30 days with stop loss 0.08
assess_risk(scu='000010.XSHE',end_date="2021-09-30",period="30",stop_loss=0.08)

In [ ]:
#Get all stocks
stocks = list(get_all_securities(['stock']).index)

In [ ]:
#select all stocks whose amplitude over the last period is less stop loss and store
#these stock names and their amplitudes into a dictionary
#@param: end_date end date of the past investment
#@param: period length of investment period(num of days)
#@return: my_stock_dic a dictionary that stores names and amplitudes of all stocks that satisfy our need 
def select_stock(end_date,period,stop_loss):
    my_stock_dic={}
    for i in stocks:
        amplitude=get_amplitude(i,end_date,period,count=1)
        if amplitude<stop_loss:
            my_stock_dic[i]=amplitude
    return my_stock_dic

In [ ]:
#log out the joinquant account
logout()